In [1]:
from scipy import io
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly
import plotly.io as pio
import Plotters
import open3d as o3d

pio.renderers.default = "browser"
path = 'J:\My Drive\Roni\csv_mov'
mov = 'mov15'
files_to_load = [f'hull_data_{mov}',f'real_coords_{mov}','rot_mat']

# load data to dictionary
data_dict = {file_to_load : pd.read_csv(f'{path}\{file_to_load}.csv') for file_to_load in files_to_load}





Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# load body and wing data to dictionary
name_of_parts = ['body','wing_l','wing_r']
fly_hull = {name_of_part : data_dict[files_to_load[0]].query(f'{name_of_part} == 1')[['x','y','z']] for name_of_part in name_of_parts}
# load real coordinates
real_coord = {ax: data_dict[files_to_load[1]].query(f'{ax} == 1')['real_coords'].reset_index(drop = True) for ax in ['x','y','z']}

In [3]:
# rotate and save the hull in lab axes
rot_mat = data_dict['rot_mat'].to_numpy()
real_hull = {body_wing : np.dot(rot_mat,np.vstack([real_coord[ax][fly_hull[body_wing][ax]] for ax in ['x','y','z']])).T for body_wing in name_of_parts}

In [6]:

Plotters.plot_hull(real_hull,size = 3)



NameError: name 'real_hull' is not defined

In [2]:
import open3d as o3d
print("Testing IO for meshes ...")
mesh = o3d.io.read_triangle_mesh("../../TestData/knot.ply")
print(mesh)
o3d.io.write_triangle_mesh("copy_of_knot.ply", mesh)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Testing IO for meshes ...
[Open3D WARNING] Read PLY failed: unable to open file: ../../TestData/knot.ply
TriangleMesh with 0 points and 0 triangles.
[Open3D WARNING] Write PLY failed: mesh has 0 vertices.


False

In [6]:

def gen_pcd(hull,color):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(hull)
    pcd.colors = o3d.utility.Vector3dVector(color)
    return pcd

In [4]:
rwing = real_hull['wing_r'][0:100,:]
pcd = o3d.geometry.PointCloud()


In [5]:
pcd.points = o3d.utility.Vector3dVector(rwing)


: 

In [8]:
from scipy import io
import numpy as np
import open3d as o3d



body = real_hull['body']
lwing = real_hull['wing_l']
rwing = real_hull['wing_r'][0:100,:]



color = np.vstack((np.tile([0.0,1.0,0.0],(body.shape[0],1)),np.tile([1.0,0.0,0.0],(rwing.shape[0],1)),np.tile([0.0,0.0,1.0],(lwing.shape[0],1))))
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(rwing)

# pcd_all= gen_pcd(body,color)
# pcd_body = gen_pcd(body,np.vstack((np.tile([0.0,1.0,0.0],(body.shape[0],1)))))
# pcd_rwing= gen_pcd(rwing,np.vstack((np.tile([0.0,1.0,0.0],(body.shape[0],1)))))
# pcd_lwing= gen_pcd(lwing,np.vstack((np.tile([0.0,1.0,0.0],(body.shape[0],1)))))
# poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_all, depth=11)[0]



: 